<a href="https://colab.research.google.com/github/rahiakela/modern-computer-vision-with-pytorch/blob/main/10-applications-of-object-detection-and-segmentation/1_multi_object_instance_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Multi-object instance segmentation

**Detectron2** is a platform built by the Facebook team. **Detectron2** includes high-quality implementations of state-of-the-art object detection algorithms, including DensePose of the Mask R-CNN model family. The original Detectron framework was written in Caffe2, while the **Detectron2** framework is written using PyTorch.

Detectron2 supports a range of tasks related to object detection. 

Like the original Detectron, it supports object detection with boxes and instance segmentation masks, as well as human pose prediction. 

Beyond that, Detectron2 adds support for semantic segmentation and panoptic segmentation (a task that combines both semantic and instance segmentation). By leveraging Detectron2, we are able to build object detection, segmentation, and pose estimation in a few lines of code.

## Setup

In [ ]:
%%shell

pip install -U torch==1.4+cu100 torchvision==0.5+cu100 -f https://download.pytorch.org/whl/torch_stable.html
pip install -qU openimages torch_snippets
pip install git+git://github.com/waspinator/pycococreator.git@0.2.0
pip install pycocotools
pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html
pip install pyyaml==5.1 pycocotools>=2.0.1

In [ ]:
from torch_snippets import *
from openimages.download import _download_images_by_id
import zipfile
import datetime
import json

from pycococreatortools import pycococreatortools
from os import listdir
from os.path import isfile, join
from PIL import Image

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.engine import DefaultTrainer
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.visualizer import ColorMode

In [ ]:
!wget -O train-annotations-object-segmentation.csv -q https://storage.googleapis.com/openimages/v5/train-annotations-object-segmentation.csv
!wget -O classes.csv -q https://raw.githubusercontent.com/openimages/dataset/master/dict.csv

## Fetching and preparing data

We will be working on the images that are available in the Open Images dataset provided by Google at https://storage.googleapis.com/openimages/web/index.html.

We will learn about fetching only the required images and not
the entire dataset. Note that this step is required, as the dataset size prohibits a typical user who might not have extensive resources from building a model.

Let's specify the classes that we want our model to predict.

In [ ]:
required_classes = "person,dog,bird,car,elephant,football,jug,laptop,Mushroom,Pizza,Rocket,Shirt,Traffic sign,Watermelon,Zebra"
required_classes = [c.lower() for c in required_classes.lower().split(",")]

classes = pd.read_csv("classes.csv", header=None)
